# XGB(one of the GLM family with Gradient Imp.): 

# Prepare the environment 

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRegressor
import optuna
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold




# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/30-days-of-ml/sample_submission.csv
/kaggle/input/30-days-of-ml/train.csv
/kaggle/input/30-days-of-ml/test.csv


# Prepare the data
data as X_train, X_valid, y_train, y_valid, **X, y, X_test**

In [2]:
train = pd.read_csv("../input/30-days-of-ml/train.csv", index_col=0)
test = pd.read_csv("../input/30-days-of-ml/test.csv", index_col=0)

train.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
id,,,,,,,,,,,,,,,,,,,,,
1,B,B,B,C,B,B,A,E,C,N,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
2,B,B,A,A,B,D,A,F,A,O,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
3,A,A,A,C,B,D,A,D,A,F,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
4,B,B,A,C,B,D,A,E,C,K,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
6,A,A,A,C,B,D,A,E,A,N,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [3]:
X_test = test.copy()
y = train['target']
X =  train.drop(['target'], axis=1).copy()

**Continuos Features Normlization**

In [4]:
def feature_norm_count(X):
    # mormlize the count X_data feature to learn faster
    pass

In [5]:
# feature_norm_count(X)

**1 - droping the bad labels data**

In [6]:

cat_cols = [col for col in X.columns if X[col].dtype == 'object']
    
good_cat_cols = [col for col in cat_cols if set(X_test[col]).issubset(set(X[col]))]
bad_cat_cols = list( set(cat_cols) - set(good_cat_cols) )

X = X.drop(bad_cat_cols, axis=1)
X_test = X_test.drop(bad_cat_cols, axis=1)

**2 - 1-hot encoding for the low cardinality data**

In [7]:
#Investing the training data
# Get number of unique entries in each column with categorical data
object_nunique = list(map(lambda col: X[col].nunique(), cat_cols))
d = dict(zip(cat_cols, object_nunique))
# print X shape
print(X.shape)
# Print number of unique entries by column, in ascending order
sorted(d.items(), key=lambda x: x[1])


(300000, 24)


[('cat0', 2),
 ('cat1', 2),
 ('cat2', 2),
 ('cat3', 4),
 ('cat4', 4),
 ('cat5', 4),
 ('cat8', 7),
 ('cat6', 8),
 ('cat7', 8),
 ('cat9', 15)]

In [8]:
d.pop('cat9')
d.pop('cat8')
un = sum([item[1] for item in d.items() ])
new = (un-1)*X.shape[0]  / 1000000
print("new entries to add:",new,"M")

new entries to add: 9.9 M


1-hot

In [9]:
cat_cols = [col for col in X.columns if X[col].dtype == 'object']
low_cardinality_cols = [col for col in cat_cols if X[col].nunique() < 8]
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

OH_cols_X = pd.DataFrame(OH_encoder.fit_transform(X[low_cardinality_cols]))
OH_cols_X_test = pd.DataFrame(OH_encoder.fit_transform(X_test[low_cardinality_cols]))

# One-hot encoding removed index; put it back
OH_cols_X.index = X.index
OH_cols_X_test.index = X_test.index

# Remove categorical columns (will replace with one-hot encoding)
num_X = X.drop(low_cardinality_cols, axis=1)
num_X_test = X_test.drop(low_cardinality_cols, axis=1)

# Add one-hot encoded columns to numerical features
X = pd.concat([num_X, OH_cols_X], axis=1)
X_test = pd.concat([num_X_test, OH_cols_X_test], axis=1)

**2 - oridnal encoding for the high cardinality data**

In [10]:
cat_cols = [col for col in X.columns if X[col].dtype == 'object']
ord_encoder = OrdinalEncoder()
X[cat_cols] = ord_encoder.fit_transform(X[cat_cols])
X_test[cat_cols] = ord_encoder.fit_transform(X_test[cat_cols])

**splitting the data**
data as X_train, X_valid, y_train, y_valid, **X, y, X_test**

In [11]:
# splitting the data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=1) 

# Features Eng.

# Tuninig

In [12]:

def tune(trial):
    learning_rate = trial.suggest_float("learning_rate", 1e-2, 0.25, log=True)
    reg_lambda = trial.suggest_loguniform("reg_lambda", 1e-8, 100.0)
    reg_alpha = trial.suggest_loguniform("reg_alpha", 1e-8, 100.0)
    subsample = trial.suggest_float("subsample", 0.1, 1.0)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.1, 1.0)
    max_depth = trial.suggest_int("max_depth", 1, 7)

    model = XGBRegressor(random_state=10,tree_method="gpu_hist",gpu_id=0,
                         predictor="gpu_predictor",n_estimators=5000,
                         learning_rate=learning_rate,reg_lambda=reg_lambda,
                         reg_alpha=reg_alpha,subsample=subsample,
                         colsample_bytree=colsample_bytree,max_depth=max_depth)
    
    model.fit(X_train, y_train, early_stopping_rounds=5, eval_set=[(X_valid, y_valid)], verbose=1000)
    y_pred = model.predict(X_valid)
    rmse = mean_squared_error(y_pred, y_valid)
    return rmse

In [13]:
study = optuna.create_study(direction="minimize")
study.optimize(tune, n_trials=500)

[I 2021-08-25 20:11:18,682] A new study created in memory with name: no-name-3a423acd-d399-4f24-9876-f4713349fec0


[0]	validation_0-rmse:7.66777
[1000]	validation_0-rmse:0.73427
[2000]	validation_0-rmse:0.73111
[3000]	validation_0-rmse:0.72931
[4000]	validation_0-rmse:0.72822
[4517]	validation_0-rmse:0.72778


[I 2021-08-25 20:11:28,957] Trial 0 finished with value: 0.5296583440306096 and parameters: {'learning_rate': 0.01412593216555015, 'reg_lambda': 1.9421985552511267, 'reg_alpha': 3.1033476887925904e-07, 'subsample': 0.37775644433800404, 'colsample_bytree': 0.9794495365507857, 'max_depth': 1}. Best is trial 0 with value: 0.5296583440306096.


[0]	validation_0-rmse:7.54164
[1000]	validation_0-rmse:0.72139
[1479]	validation_0-rmse:0.71955


[I 2021-08-25 20:11:33,384] Trial 1 finished with value: 0.5177465280371999 and parameters: {'learning_rate': 0.030498812721195052, 'reg_lambda': 2.129275447791715e-06, 'reg_alpha': 0.00026114627207218485, 'subsample': 0.29948588742744714, 'colsample_bytree': 0.26038057138004533, 'max_depth': 3}. Best is trial 1 with value: 0.5177465280371999.


...

[I 2021-08-25 20:56:03,631] Trial 497 finished with value: 0.5141281032935037 and parameters: {'learning_rate': 0.09590482562913358, 'reg_lambda': 6.297864958598139e-06, 'reg_alpha': 41.36082665095755, 'subsample': 0.9614981363897572, 'colsample_bytree': 0.2420613694375489, 'max_depth': 5}. Best is trial 445 with value: 0.5130462470767222.


[0]	validation_0-rmse:7.14750
[769]	validation_0-rmse:0.71642


[I 2021-08-25 20:56:07,757] Trial 498 finished with value: 0.5132512075675807 and parameters: {'learning_rate': 0.08165617388720212, 'reg_lambda': 3.1413443878923315e-05, 'reg_alpha': 27.794200873922406, 'subsample': 0.9657630230561375, 'colsample_bytree': 0.1760438260766778, 'max_depth': 5}. Best is trial 445 with value: 0.5130462470767222.


[0]	validation_0-rmse:7.17904
[756]	validation_0-rmse:0.71680


[I 2021-08-25 20:56:11,739] Trial 499 finished with value: 0.513800268144175 and parameters: {'learning_rate': 0.07756037325689734, 'reg_lambda': 1.9111366383486403e-05, 'reg_alpha': 27.890169870497107, 'subsample': 0.9635192901519983, 'colsample_bytree': 0.19194437919120627, 'max_depth': 5}. Best is trial 445 with value: 0.5130462470767222.


In [14]:
best_para = study.best_params
print(best_para)

{'learning_rate': 0.09739156815144875, 'reg_lambda': 1.9383336479859923e-05, 'reg_alpha': 24.134445292436887, 'subsample': 0.9936134241222736, 'colsample_bytree': 0.16484879725660215, 'max_depth': 5}


# Train

In [15]:
# # Define the model
# model = XGBRegressor(random_state = 10, n_estimators = 5000, n_jobs =4)#,**best_para)

# # Fit the model
# model.fit(X_train, y_train, early_stopping_rounds = 10, eval_set=[(X_valid, y_valid)], verbose = 100)


# validation


In [16]:
# # RMSE-based(no validatio n)
# y_pred = model.predict(X_valid)
# rmse = mean_squared_error(y_pred, y_valid)
# print(rmse)

# Another train: Kfolding

# predictions 

In [17]:
# model.fit(X, y, early_stopping_rounds = 10, eval_set=[(X_valid, y_valid)], verbose = False)
# predictions = model.predict(X_test)

# Submit

In [18]:

# # Save the predictions to a CSV file
# output = pd.DataFrame({'Id': X_test.index,
#                        'target': predictions})
# output.to_csv('submission.csv', index=False)